# 方案说明
本方案是一个简单的根据0基础入门的示例中的识别手写数字改写而成的黄斑定位程序。<br />
本方案从头到尾直接运行即可，整体布局为 **数据获取 -> 函数定义 -> 训练 -> 预测**。<br />

# 题目分析
## 难点<br />
&ensp;&ensp;题目要求为在一组图片上找到黄斑中心，但是图片的大小不一，每张图片对应的黄斑中心的取值范围也随着图片大小的变化而变化，从而缺乏一个统一的标准。
    
## 解决方案<br />
&ensp;&ensp;指定缩放尺寸，将所有的图片均缩放到指定尺寸。对于中心点坐标，将所有的中心点坐标转化为中心点在xy轴的百分比位置，如目标点在最中间则认为横纵分别为0.5。最后再将预测后的小数还原为整数即可得到真实图片上的像素点坐标。

## 具体实现策略<br />
&ensp;&ensp;（本程序没有实现异步读取，直接做了最基础的读数据，运行。最后会在./work/Fovea_Localization_Results.csv中保存测试结果。）<br />
&ensp;&ensp;1. 依次读取训练图片，将图片转为灰度模式，将图片缩放到指定尺寸，将黄斑中心转化到[0,1]。直接使用dataframe记录图片信息和黄斑中心分别作为输入和输出。<br />
&ensp;&ensp;2. 构造神经网络，这部分直接参考0基础入门的示例中的识别手写数字。[https://www.paddlepaddle.org.cn/tutorials/projectdetail/2182025]<br />
&ensp;&ensp;3. 训练数据，将之前保存过的dataframe转化为tensor直接训练即可，本程序将xy坐标分开进行训练。<br />
&ensp;&ensp;4. 依次读取测试图片，将图片转为灰度模式，将图片缩放到指定尺寸。直接使用dataframe记录图片信息和原始图片尺寸，原始图片尺寸用于将预测结果转化为真实坐标。<br />
&ensp;&ensp;5. 预测，并将预测后的值转化到真实坐标上。

# 程序开始
## 获取数据

In [1]:
# 下载数据
import urllib 
import requests   
import os
url = 'https://bj.bcebos.com/v1/dataset-bj/%E5%8C%BB%E7%96%97%E6%AF%94%E8%B5%9B/%E5%B8%B8%E8%A7%84%E8%B5%9B%EF%BC%9APALM%E7%9C%BC%E5%BA%95%E5%BD%A9%E7%85%A7%E4%B8%AD%E9%BB%84%E6%96%91%E4%B8%AD%E5%A4%AE%E5%87%B9%E5%AE%9A%E4%BD%8D.zip'  

if not os.path.exists('./work/Train_and_test.zip'):
    print("Downloading start!")
    urllib.request.urlretrieve(url, "./work/Train_and_test.zip")  
    print("Downloading end!")
else:
    print("Already Downloading")

In [2]:
# 解压数据
! unzip -oq ./work/Train_and_test.zip -d ./work

## 定义（包，变量，函数）<br />

自定义变量包括:<br />
&ensp;&ensp;image_size 		 # 缩放图片的大小<br />
&ensp;&ensp;m_ite	   		# 训练的迭代次数，相当于基础教程里的epoch<br />
&ensp;&ensp;model_save_dir	 # 训练后的模型保存地址<br />

自定义函数包括:<br />
&ensp;&ensp;get_train_image		读取训练图片，缩放图片到指定大小<br />
&ensp;&ensp;get_test_image		读取测试图片，缩放图片到指定大小<br />
&ensp;&ensp;Mymodel				定义CNN网络<br />
&ensp;&ensp;train				对网络进行训练<br />
&ensp;&ensp;test				对网络进行测试并且写入.csv<br />

In [3]:
import paddle
import paddle.nn.functional as F
import os
import numpy as np
from paddle.nn import Conv2D, MaxPool2D, Linear
import pandas as pd
from PIL import Image
import random

# 定义缩放的大小，训练迭代次数，模型保存目录
global image_size
image_size = 96
m_ite=5000
model_save_dir='./work/mymodel'
last_model_dir='./work/mymodel'

def get_train_image(image_size):
    pd.set_option('mode.chained_assignment', None)
    # 读xlsx
    df = pd.read_excel('./work/常规赛：PALM眼底彩照中黄斑中央凹定位/Train/Fovea_Location_train.xlsx')
    # 定义变量用于保存图片的原始大小和修改后的中心点坐标
    df['ori_width'] = 0
    df['ori_height'] = 0
    df['changed_x'] = 0.0
    df['changed_y'] = 0.0
    im_list = []
    for i in range(len(df)):  # df[df.columns[0]]:
        if i % 50 == 0:
            print(str(i / len(df) * 100) + '% finished')
        im = Image.open('./work/常规赛：PALM眼底彩照中黄斑中央凹定位/Train/fundus_image/' + df[df.columns[0]][i])
        df['ori_width'][i] = np.array(im).shape[1]  # 列对应横向宽度，行对应纵向宽度
        df['ori_height'][i] = np.array(im).shape[0]
        df['changed_x'][i] = df['Fovea_X'][i]/np.array(im).shape[1]
        df['changed_y'][i] = df['Fovea_Y'][i]/np.array(im).shape[0]
        #转化为灰度图并缩小，保存在列表中
        im = im.convert('L')
        im = im.resize((image_size, image_size), Image.ANTIALIAS)
        im = np.array(im).reshape(1, -1).astype(np.float32)
        im_list.append(im.tolist()[0])
    print(str(1 * 100) + '% finished')
    im_record = np.array(im_list)
    df2 = pd.DataFrame(im_record)
    pd.set_option('mode.chained_assignment', "raise")
    # return train_infor, train_input
    # 返回图片的标签信息和图片向量
    return df, df2

def get_test_image(image_size):
    # read the test_infor
    mylist=[]
    im_list = []
    for i in range(400):
        if i % 50 == 0:
            print(str(i / 400 * 100) + '% finished')
        target_pic_name='T' + ('%04d' % (i+1)) + '.jpg'
        # 无法直接从csv中读取，所以手动生成向量
        tmplist=[target_pic_name,0.0,0.0,0,0,0.0,0.0]
        im = Image.open('./work/常规赛：PALM眼底彩照中黄斑中央凹定位/PALM-Testing400-Images/' + target_pic_name)
        tmplist[3] = np.array(im).shape[1]  # 列对应横向宽度，行对应纵向宽度
        tmplist[4] = np.array(im).shape[0]
        im = im.convert('L')
        im = im.resize((image_size, image_size), Image.ANTIALIAS)
        im = np.array(im).reshape(1, -1).astype(np.float32)
        im_list.append(im.tolist()[0])

        mylist.append(tmplist)
    print(str(1 * 100) + '% finished')
    test_df = pd.DataFrame.from_records(mylist, columns=['FileName', 'Fovea_X', 'Fovea_Y', 'ori_width', 'ori_height', 'changed_x', 'changed_y'])
    im_record = np.array(im_list)
    df_test_input = pd.DataFrame(im_record)
    pd.set_option('mode.chained_assignment', "raise")
    # return test_infor, test_input
    return test_df, df_test_input

#定义模型，本模型参考了https://www.paddlepaddle.org.cn/tutorials/projectdetail/2182025
class Mymodel(paddle.nn.Layer):
    def __init__(self):
        super(Mymodel, self).__init__()
        global image_size
        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv1 = Conv2D(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv2 = Conv2D(in_channels=20, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义全连接层，输出维度是2
        # 计算维度
        tmp=np.zeros([image_size,image_size], dtype='float32', order='C')
        tmp = np.array(tmp).reshape(1, 1, image_size, image_size).astype(np.float32)
        tmp=paddle.to_tensor(tmp)
        tmp1 = self.conv1(tmp)
        tmp2 = self.max_pool1(tmp1)
        tmp3 = self.conv2(tmp2)
        tmp4 = self.max_pool2(tmp3)
        liner_input_num=1
        for i in range(len(tmp4.shape)):
            liner_input_num*=tmp4.shape[i]
        self.fc = Linear(in_features=liner_input_num, out_features=5000)
        self.fc2 = Linear(in_features=5000, out_features=1000)
        self.fc3 = Linear(in_features=1000, out_features=1)

    # 定义网络前向计算过程，卷积后紧接着使用池化层，最后使用全连接层计算最终输出
    def forward(self, inputs):
         x = self.conv1(inputs)
         x = F.relu(x)
         x = self.max_pool1(x)
         x = self.conv2(x)
         x = F.relu(x)
         x = self.max_pool2(x)
         x = paddle.reshape(x, [x.shape[0], -1])
         x = self.fc(x)
         x = F.relu6(x)
         x =self.fc2(x)
         x = F.tanh(x)
         x =self.fc3(x)
         return x

# 定义训练
def train(model1,model2, train_infor, train_input,eval_infor,eval_input,image_size,m_ite):
    # 将数据转化为四维矩阵格式，并归一到0-1
    train_im = paddle.to_tensor(train_input.values.astype('float32')/255)
    train_im = paddle.reshape(train_im,[train_im.shape[0], 1, image_size, image_size])
    train_lab_x = train_infor[['changed_x']]
    train_lab_x = paddle.to_tensor(train_lab_x.values.astype('float32'))
    train_lab_y = train_infor[['changed_y']]
    train_lab_y = paddle.to_tensor(train_lab_y.values.astype('float32'))

    eval_im = paddle.to_tensor(eval_input.values.astype('float32')/255)
    eval_im = paddle.reshape(eval_im,[eval_im.shape[0], 1, image_size, image_size])
    eval_lab_x = eval_infor[['changed_x']]
    eval_lab_x = paddle.to_tensor(eval_lab_x.values.astype('float32'))
    eval_lab_y = eval_infor[['changed_y']]
    eval_lab_y = paddle.to_tensor(eval_lab_y.values.astype('float32'))

    # 定义学习器
    opt1 = paddle.optimizer.Adam(learning_rate=0.001, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),
                                parameters=model1.parameters())
    opt2 = paddle.optimizer.Adam(learning_rate=0.001, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),
                                parameters=model2.parameters())

    print('init train!')
    
    best_loss_x=1000
    best_loss_y=1000
    best_x_ite=-1
    best_y_ite=-1

    # 训练
    for i in range(m_ite):
        predicts1 = model1(train_im)
        predicts2 = model2(train_im)
        loss1 = F.square_error_cost(predicts1, train_lab_x)
        loss2 = F.square_error_cost(predicts2, train_lab_y)
        avg_loss1 = paddle.mean(loss1)
        avg_loss2 = paddle.mean(loss2)

        avg_loss1.backward()
        avg_loss2.backward()
        opt1.step()
        opt1.clear_grad()
        opt2.step()
        opt2.clear_grad()
        
        model1.eval()
        model2.eval()

        predictsx = model1(eval_im)
        predictsy = model2(eval_im)
        lossx = F.square_error_cost(predictsx, eval_lab_x)
        lossy = F.square_error_cost(predictsy, eval_lab_y)
        avg_lossx = paddle.mean(lossx)
        avg_lossy = paddle.mean(lossy)
        
        if avg_lossx.numpy()<best_loss_x:
            best_loss_x=avg_lossx.numpy()
            paddle.save(model1.state_dict(), model_save_dir+'x')
            best_x_ite=i
        if avg_lossy.numpy()<best_loss_y:
            best_loss_y=avg_lossy.numpy()
            paddle.save(model2.state_dict(), model_save_dir+'y')
            best_y_ite=i
        
        model1.train()
        model2.train()
        
        if i%100==0:
            print("ite: {}, x los: {}, y los: {}, best x ite: {}, best y ite: {}".format(i,avg_loss1.numpy(),avg_loss2.numpy(),best_x_ite,best_y_ite))

#定义测试
def test(model1, model2, test_infor, test_input ,image_size):
    print('test_start')
    # param_dict = paddle.load('./PALM')
    # model.load_dict(param_dict)
    
    model1.eval()
    model2.eval()

    test_im = paddle.to_tensor(test_input.values.astype('float32')/255)
    test_im = paddle.reshape(test_im,[test_im.shape[0], 1, image_size, image_size])
    predicts1 = model1(test_im)
    predicts1 = predicts1.numpy()
    predicts2 = model2(test_im)
    predicts2 = predicts2.numpy()

    pd.set_option('mode.chained_assignment', None)

    #将预测的点还原为图片上的坐标
    for i in range(400):
        test_infor['changed_x'][i]=predicts1[i][0]
        test_infor['changed_y'][i]=predicts2[i][0]
        test_infor['Fovea_X'][i] = predicts1[i][0]*test_infor['ori_width'][i]
        test_infor['Fovea_Y'][i] = predicts2[i][0]*test_infor['ori_height'][i]
    pd.set_option('mode.chained_assignment', 'raise')
    
    #写文件
    final_df=test_infor[['FileName','Fovea_X','Fovea_Y']]
    final_df.to_csv('./work/Fovea_Localization_Results.csv', index=None)

## 正式进行训练和测试

In [11]:
#获得训练数据
print("get_train_image!")
train_infor, train_input=get_train_image(image_size)
print("train infor gotten!")

train_infor.to_csv('train_infor.csv',index=None)
train_input.to_csv('train_input.csv',index=None)

get_train_image!
0.0% finished
6.25% finished
12.5% finished
18.75% finished
25.0% finished
31.25% finished
37.5% finished
43.75% finished
50.0% finished
56.25% finished
62.5% finished
68.75% finished
75.0% finished
81.25% finished
87.5% finished
93.75% finished
100% finished
train infor gotten!


In [12]:
train_infor=pd.read_csv('train_infor.csv')
train_input=pd.read_csv('train_input.csv')

train_num=len(train_infor)
eval_percent=0.2;
cut_point=int(train_num*eval_percent)

index=list(range(train_num))
random.shuffle(index)

eval_infor=train_infor.iloc[index[:cut_point],:]
eval_input=train_input.iloc[index[:cut_point],:]
train_infor=train_infor.iloc[index[cut_point:],:]
train_input=train_input.iloc[index[cut_point:],:]

In [13]:
# 根据测试，如果目标得到最优xy而非对样本的无尽拟合，200次迭代足够了
m_ite=5000

#构造模型开始训练
model1 = Mymodel()
model2 = Mymodel()
print("model created")
train(model1,model2,train_infor, train_input,eval_infor,eval_input,image_size, m_ite)
print("train finish")
paddle.save(model1.state_dict(), model_save_dir+str(1))
paddle.save(model2.state_dict(), model_save_dir+str(2))
print("model saved")

model created
init train!
ite: 0, x los: [0.4707266], y los: [0.44537887], best x ite: -1, best y ite: -1
ite: 100, x los: [0.00702699], y los: [0.00342633], best x ite: 93, best y ite: 73
ite: 200, x los: [0.00246863], y los: [0.00237697], best x ite: 149, best y ite: 73
ite: 300, x los: [0.00122925], y los: [0.00207924], best x ite: 149, best y ite: 73
ite: 400, x los: [0.00073432], y los: [0.00170266], best x ite: 149, best y ite: 73
ite: 500, x los: [0.00097957], y los: [0.00152692], best x ite: 149, best y ite: 73
ite: 600, x los: [0.00043496], y los: [0.00146891], best x ite: 149, best y ite: 73
ite: 700, x los: [0.00032016], y los: [0.00142753], best x ite: 149, best y ite: 73
ite: 800, x los: [0.0003557], y los: [0.00139217], best x ite: 149, best y ite: 73
ite: 900, x los: [0.00028052], y los: [0.00135927], best x ite: 149, best y ite: 73
ite: 1000, x los: [0.00020784], y los: [0.00132983], best x ite: 149, best y ite: 73
ite: 1100, x los: [0.00022118], y los: [0.00130157], be

In [14]:
test_infor, test_input=get_test_image(image_size)
test_infor.to_csv('test_infor.csv',index=None)
test_input.to_csv('test_input.csv',index=None)

0.0% finished
12.5% finished
25.0% finished
37.5% finished
50.0% finished
62.5% finished
75.0% finished
87.5% finished
100% finished


In [15]:
#开始测试
model1 = Mymodel()
model2 = Mymodel()
print("model created")
param_dict1 = paddle.load(last_model_dir+'x')
model1.load_dict(param_dict1)
param_dict2 = paddle.load(last_model_dir+'y')
model2.load_dict(param_dict2)

test(model1,model2, test_infor, test_input ,image_size)

model created
test_start


# 总结和一些讨论
相对于之前的工作增加了验证集，并且将xy分开进行处理，但是提升的效果仍为有限，可见想要提高效果从网络结构下手更好。
<br />